In [111]:
import torch
from torch.utils.data import Dataset,DataLoader
import numpy as np
import spacy

In [167]:
class MyDataSet(Dataset):
    def __init__(self,path,exts=('.de','.en'),maxLen=100):
        with open(path) as fs:
            ds=[]
            for line in fs:
                ds.append(line.strip())
        self._data=list(filter(lambda x:len(x)<maxLen,ds))
    def __getitem__(self,idx):
        if isinstance(idx,int):
            return self._data[idx].strip()
        if isinstance(idx,slice):
            if idx.start>=len(self._data):
                raise StopIteration
            return self._data[idx.start:idx.stop]
    def __len__(self):
        return len(self._data)
class MyDataLoader():
    def __init__(self,ds,batch_size,batch_size_fn=None):
        self.ds=ds
        self.batch_size=batch_size
        self.pos=0
        self.maxqueue=128
        self.toSmall=int(self.maxqueue/4)
        self.batch_size_fn=batch_size_fn
        self.buffer=[]
    def __iter__(self):
        return self
    def __next__(self):        
        cnt=1
        currentBatchSize=0    
        return_example=[]
        
        if len(self.buffer)<self.toSmall:
            if self.pos<len(self.ds):
                fill_amount=self.maxqueue-len(self.buffer)
                self.buffer.extend(self.ds[self.pos:self.pos+fill_amount])
                self.pos+=fill_amount
            elif len(self.buffer)==0:
                self.pos=0
                raise StopIteration
                
        cnt,currentBatchSize=1,0     
        for example in self.buffer:
            currentBatchSize=self.batch_size_fn(example,cnt,currentBatchSize)
            cnt+=1
            if currentBatchSize>self.batch_size:break
            return_example.append(example)
        for _ in range(len(return_example)):
            self.buffer.pop(0)
            
        return return_example
maxSeqLen=0
def batch_size_fn(new,count,validbatch):
    global maxSeqLen
    if count==1:maxSeqLen=0
    maxSeqLen=max(maxSeqLen,len(new))
    return maxSeqLen*count

In [168]:
ds=MyDataSet('.data/iwslt/de-en/IWSLT16.TED.dev2010.de-en.en',maxLen=100000)
loader=MyDataLoader(ds,batch_size=1000,batch_size_fn=batch_size_fn)

c=0
for x in loader:
#     print(x)
    c+=len(x)
print(c)

887


In [169]:
c=0
for x in loader:
#     print(x)
    c+=len(x)
print(c)
print(x)

887
["And today, I had time to show you one point in this new design space, and a few of the possibilities that we're working to bring out of the laboratory.", "So the thought I want to leave you with is that we're on the cusp of this new generation of tools for interacting with digital media that are going to bring information into our world on our terms.", 'Thank you very much.', 'I look forward to talking with all of you.']


In [164]:
! wc -l .data/iwslt/de-en/IWSLT16.TED.dev2010.de-en.en 

887 .data/iwslt/de-en/IWSLT16.TED.dev2010.de-en.en


In [121]:
a=slice(10,20)

In [85]:
a.step